### Knapsack Problem

> Maximize
 $$\sum_{i\in I} p_i*x_i$$
> Sujeito a:
$$ \sum_{i \in I}w_i*x_i \le c$$
$$x_i \in \{0,1\} \forall_i \in I

In [6]:
from mip import Model, xsum, maximize, BINARY

p = [10, 13, 18, 31, 7, 15]
w = [11, 15, 20, 35, 10, 33]
c, I = 47, range(len(w))

m = Model('knapsack')

x = [m.add_var(var_type = BINARY) for i in I]

m.objective = maximize(xsum(p[i] * x[i] for i in I))

m += xsum(w[i] * x[i] for i in I)<= c

m.optimize()

selected = [i for i in I if x[i].x >= 0.99]
print("Selected items: {}".format(selected))

Selected items: [0, 3]


### TSP

> Minimize:
$$\sum_{i \in I, j \in I} c_{i,j} * x_{i,j}$$

> Sujeito a:
$$ \sum_{j \in V\\\{i\}} x_{i,j} = 1 \; \forall i \in V$$
$$ \sum_{i \in V\\\{j\}} x_{i,j} = 1 \; \forall j \in V$$
$$ y_i - (n+1) \geq y_i - n \;\;\forall i \in V\\\{0\}, j \in V\{0,1\}$$
$$ x_{i,j} \in \{0,1\} \; \forall i \in V, j \in V$$
$$ y_i \geq 0 \forall i \in V$$



In [12]:
from itertools import product
from sys import stdout as out 
from mip import Model, xsum, minimize, BINARY 

# names of places to visit 
places = ['Antwerp', 'Bruges', 'C-Mine', 'Dinant', 'Ghent',
          'Grand-Place de Bruxelles', 'Hasselt', 'Leuven',
          'Mechelen', 'Mons', 'Montagne de Bueren', 'Namur',
          'Remouchamps', 'Waterloo']

# distances in an upper triangular matrix
dists = [[83, 81, 113, 52, 42, 73, 44, 23, 91, 105, 90, 124, 57],
         [161, 160, 39, 89, 151, 110, 90, 99, 177, 143, 193, 100],
         [90, 125, 82, 13, 57, 71, 123, 38, 72, 59, 82],
         [123, 77, 81, 71, 91, 72, 64, 24, 62, 63],
         [51, 114, 72, 54, 69, 139, 105, 155, 62],
         [70, 25, 22, 52, 90, 56, 105, 16],
         [45, 61, 111, 36, 61, 57, 70],
         [23, 71, 67, 48, 85, 29],
         [74, 89, 69, 107, 36],
         [117, 65, 125, 43],
         [54, 22, 84],
         [60, 44],
         [97],
         []]

# number of nodes and list of vertices
n, V = len(dists), set(range(len(dists)))

# distances matrix
c = [[0 if i == j
      else dists[i][j-i-1] if j > i
      else dists[j][i-j-1] 
      for j in V] for i in V]

model = Model()

# binary variables indicating if arc(i,j) is used on the route or not
x = [[model.add_var(var_type = BINARY) for j in V] for i in V]

# continuous variable to prevent subtours: each city will have a
# different sequential id in the planned route except the first one
y = [model.add_var() for i in V]

# objective function: minimize the distance
model.objective = maximize(xsum(c[i][j]*x[i][j] for i in V for j in V))

# constraint: Leave each city only once
for i in V:
    model += xsum(x[i][j] for j in V - {i}) == 1

# constraint: Enter each city only once
for i in V:
    model += xsum(x[j][i] for j in V - {i}) == 1

# constraint: subtour elimination
for(i, j) in product(V - {0}, V - {0}):
    if i != j:
        model += y[i] - (n + 1)*x[i][j] >= y[j] - n 

#Optimizing
model.optimize()

# Checking if a solution was found 
if model.num_solutions:
    out.write('route with total distance %g found: %s'
              %(model.objective_value, places[0]))
    nc = 0
    while True:
        nc = [i for i in V if x[nc][i].x >= 0.99][0]
        out.write(' -> %s' %places[nc])
        if nc == 0:
            break
    out.write('\n')    

route with total distance 1455 found: Antwerp -> Namur -> Ghent -> Hasselt -> Mons -> Leuven -> Waterloo -> C-Mine -> Grand-Place de Bruxelles -> Montagne de Bueren -> Bruges -> Remouchamps -> Mechelen -> Dinant -> Antwerp


### Resource Constrained Project Scheduling
>Minimize:
$$\sum_{t \in T} t*x_{(n+1, t)}$$

>Sujeito a:
$$\sum_{t \in T} x_{(j,t)} = 1 \; \forall j \in J$$
$$\sum_{j \in J}\sum_{t_2 = t - p_j + 1}^t u_{j,r}x_{j, t_2} \leq c_r \; \forall t \in T, \; r \in R$$
$$\sum_{t \in T} t*x_{s,t} - \sum_{t \in T} t*x_{j,t} \geq p_j \; \forall (j,s) \in S$$
$$x_{(j,t)} \in \{0,1\} \; \forall j \in J, t \in T$$

In [28]:
from itertools import product 
from mip import Model, xsum, BINARY 

n = 10 

p = [0, 3, 2, 5, 4, 2, 3, 4, 2, 4, 6, 0]

u = [[0, 0], [5, 1], [0, 4], [1, 4], [1, 3], [3, 2], [3, 1], [2, 4],
     [4, 0], [5, 2], [2, 5], [0, 0]]

c = [6, 8]

S = [[0, 1], [0, 2], [0, 3], [1, 4], [1, 5], [2, 9], [2, 10], [3, 8], [4, 6],
     [4, 7], [5, 9], [5, 10], [6, 8], [6, 9], [7, 8], [8, 11], [9, 11], [10, 11]]

(R, J, T) = (range(len(c)), range(len(p)), range(sum(p)))

model = Model()

x = [[model.add_var(name = "x({}, {})".format(j, t), var_type = BINARY) for t in T] for j in J]

for j in J:
    model += xsum(x[j][t] for t in T) == 1 

for(r, t) in product(R,T):
    model += (
        xsum(u[j][r] * x[j][t2] for j in J for t2 in range(max(0, t - p[j] + 1), t + 1))
        <= c[r]
    )

for (j, s) in S:
    model += xsum(t * x[s][t] - t * x[j][t] for t in T) >= p[j]

model.optimize()

print('Schedule: ')
for (j, t) in product(J, T):
    if x[j][t].x >= 0.99:
        print("Job {}: begins at t={} and finishes at t={}".format(j, t, t + p[j]))
print('Makespan = {}'.format(model.objective_value))        

Schedule: 
Job 0: begins at t=6 and finishes at t=6
Job 1: begins at t=7 and finishes at t=10
Job 2: begins at t=13 and finishes at t=15
Job 3: begins at t=26 and finishes at t=31
Job 4: begins at t=12 and finishes at t=16
Job 5: begins at t=11 and finishes at t=13
Job 6: begins at t=23 and finishes at t=26
Job 7: begins at t=16 and finishes at t=20
Job 8: begins at t=32 and finishes at t=34
Job 9: begins at t=26 and finishes at t=30
Job 10: begins at t=20 and finishes at t=26
Job 11: begins at t=34 and finishes at t=34
Makespan = 0.0


In [19]:
p = [0, 3, 2, 5, 4, 2, 3, 4, 2, 4, 6, 0]

u = [[0, 0], [5, 1], [0, 4], [1, 4], [1, 3], [3, 2], [3, 1], [2, 4],
     [4, 0], [5, 2], [2, 5], [0, 0]]

c = [6, 8]

S = [[0, 1], [0, 2], [0, 3], [1, 4], [1, 5], [2, 9], [2, 10], [3, 8], [4, 6],
     [4, 7], [5, 9], [5, 10], [6, 8], [6, 9], [7, 8], [8, 11], [9, 11], [10, 11]]

#(R, J, T) = (range(len(c), range(len(p)), range(sum(p))))

In [26]:
range(len(c)), range(len(p)), range(sum(p))

(range(0, 2), range(0, 12), range(0, 35))

### Problemas Listas

In [194]:
from itertools import repeat, product
from mip import Model, xsum, BINARY, maximize
import pandas as pd

In [114]:
model = Model()

valores = list(repeat(1, 7)) + list(repeat(0.5, 7)) + list(repeat(0, 7))
coef = dict(zip(range(21), valores))
volume = {0:3.5, 1:3.5, 2:3.5}
garrafas = {0:7, 1:7, 2:7}

x = [[model.add_var(name = "x({}, {})".format(j, t), var_type = BINARY) for t in range(3)] for j in range(21)]
v = [model.add_var(name = "v{}".format(i)) for i in range(3)]

for i in range(3):
    model += xsum([x[j][i]*coef[j] for j in range(21)]) == volume[i] 

for i in range(3):
    model += xsum([x[j][i] for j in range(21)]) == 7


for i in range(21):
    model += xsum([x[i][j] for j in range(3)]) <= 1   


model.optimize()

dados = {}
individuos = list()
volumes = list()
item = []
print('Distribuição de Garrafas: ')
for (i, j) in product(range(21), range(3)):
    if(x[i][j].x > 0.99):
        print(f'Individuo {j}\t volume: {coef[i]}: {x[i][j].x}')
        individuos.append(j)
        volumes.append(coef[i])
        item.append(x[i][j].x)
dados['individuo'] = individuos
dados['item'] = item
dados['volume'] = volumes

df = pd.DataFrame.from_dict(dados)
df = df.sort_values('individuo')
df.groupby(['individuo', 'volume']).sum('item')

Distribuição de Garrafas: 
Individuo 2	 volume: 1: 1.0
Individuo 0	 volume: 1: 1.0
Individuo 0	 volume: 1: 1.0
Individuo 0	 volume: 1: 1.0
Individuo 2	 volume: 1: 1.0
Individuo 1	 volume: 1: 1.0
Individuo 1	 volume: 1: 1.0
Individuo 1	 volume: 0.5: 1.0
Individuo 0	 volume: 0.5: 1.0
Individuo 2	 volume: 0.5: 1.0
Individuo 2	 volume: 0.5: 1.0
Individuo 1	 volume: 0.5: 1.0
Individuo 2	 volume: 0.5: 1.0
Individuo 1	 volume: 0.5: 1.0
Individuo 0	 volume: 0: 1.0
Individuo 1	 volume: 0: 1.0
Individuo 2	 volume: 0: 1.0
Individuo 0	 volume: 0: 1.0
Individuo 1	 volume: 0: 1.0
Individuo 2	 volume: 0: 1.0
Individuo 0	 volume: 0: 1.0


item
individuo volume      
0         0.0      3.0
          0.5      1.0
          1.0      3.0
1         0.0      2.0
          0.5      3.0
          1.0      2.0
2         0.0      2.0
          0.5      3.0
          1.0      2.0

In [248]:
# Definicao das variaveis
Custo = dict(zip(range(1,8), [3.6, 2.3, 4.1, 3.15, 2.8, 2.65, 3.1]))
Pop = dict(zip(range(1,16), [4, 3, 10, 14, 6, 7, 9, 10, 13, 11, 6, 12, 7, 5, 16]))
cobertura = dict(zip(range(1,8), [[1,2], [2,3,5], 
                    [1,7,9,10], [4, 6, 8, 9], [6, 7, 9, 11], [5, 7, 10, 12, 14], [12, 13, 14, 15]]))

# Calculo da populacao atendida por transmissora
n_max = len(Custo) + 1

cobertura_pop = dict()
for j in cobertura.keys():
    p = 0
    for c in cobertura[j]:
        p += Pop[c]
    cobertura_pop[j] = p

# Inicializacao do modelo
model = Model()

x = [model.add_var(name = "x({})".format(i),var_type = BINARY) for i in range(1,9)]

# Funcao Objetivo: Maximizar o atendimento a populacao
model.objective = maximize(xsum(x[i]*cobertura_pop[i] for i in range(1,8)))

# Restricao de investimento
model += xsum(x[i]*Custo[i] for i in range(1,8)) <= 15

model.optimize()

print('Transmissoras: ')
for i in range(1,8):
    if x[i].x >= 0.99:
         print(f"Transmissora {i}")
print('Populacao atendida = {} mil habitantes'.format(model.objective_value))  

model.write('ex_2.lp')

Transmissoras: 
Transmissora 2
Transmissora 4
Transmissora 5
Transmissora 6
Transmissora 7
Populacao atendida = 181.0 mil habitantes


In [250]:
compartimento = [500, 750, 1200, 1500, 1750]
capacidades = dict(zip(range(1,5), repeat(compartimento, 4)))
capacidades

{1: [500, 750, 1200, 1500, 1750],
 2: [500, 750, 1200, 1500, 1750],
 3: [500, 750, 1200, 1500, 1750],
 4: [500, 750, 1200, 1500, 1750]}

In [263]:
capacidades

{1: [500, 750, 1200, 1500, 1750],
 2: [500, 750, 1200, 1500, 1750],
 3: [500, 750, 1200, 1500, 1750],
 4: [500, 750, 1200, 1500, 1750]}

In [275]:
compartimento = [500, 750, 1200, 1500, 1750]
capacidades = dict(zip(range(5), repeat(compartimento, 4)))
demandas = {1: 10_000, 2: 15_000, 3: 12_000, 4: 8_000}
custo = {1:5, 2: 12, 3: 8, 4: 10}

model = Model()

x = [[model.add_var(name = "x({},{})".format(i, j), var_type = BINARY) for i in range(5)] for j in range(1,6)]

# Funcao Objetivo: Maximizar o atendimento a populacao
model.objective = maximize(xsum(x[i][j] for i in range(5) for j in range(1,5)))



model.write('ex_3.lp')